In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyodbc

     |████████████████████████████████| 280 kB 8.9 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287325 sha256=38bdcc962625403b2402ecb071bb9c7471c8b8f56719ff214739f91ea5b8c5d3
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc


In [3]:
!pip install eurostat

     |████████████████████████████████| 45 kB 2.0 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15147 sha256=8290e80b1666955ca185525cec5e23fb4e9bbd2497208536f9f9e3e78bc231eb
  Stored in directory: /root/.cache/pip/wheels/58/88/2a/8d619cf38d7cf939e54b6ccdece05d31b64b3eb419c11d1ed3
Successfully built jsonpath-rw


In [4]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to

In [5]:
import gensim
import pandas as pd
import numpy as np
import re
import pyodbc
import re

In [6]:
c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')

In [7]:
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")
cursor = c.cursor()

In [8]:
def load_table(cursor,query):
  cursor.execute(query)
  t1 = cursor.fetchall()
  df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
  return df



##Load Terms-Topics-Link Info
query =      """SELECT term_id,term_name FROM ESTAT.V1.tm_terms"""

termsEurostat = load_table(cursor,query)

query =      """SELECT topic_id,topic_keywords,topic_name FROM ESTAT.V1.tm_topics"""

topicsEurostat = load_table(cursor,query)

In [9]:
##Insert Topics
f = open("termTopicType.ttl", "a")
for name, row in topicsEurostat.iterrows():
  helperTopic = ""
  keywords = row[1].split(", ")
  title = str(row[2]).replace(" ", "_").replace(",", "").lower()


  prehelper = "estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', title).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_"))  + \
  " rdf:type estat:Topic.\n" + "estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', title).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) + \
  " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n"
  helper = ""
  for k in keywords:
    helper = helper +  "estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', title).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) + \
    " estat:keyword \"" + re.sub("[^a-zA-Z\d%]", ' ', k).replace("\'", "").replace("\"", "").replace("\\", "").strip() + "\"^^xsd:string.\n"
  
  helperTopic = prehelper + helper
  f.write(helperTopic)


##Insert Type
helperType = """
estatdata:var rdf:type estat:ConceptType.\n
estatdata:var estat:title \"VAR\"^^xsd:string.\n
estatdata:term rdf:type estat:ConceptType.\n
estatdata:term estat:title \"TERM\"^^xsd:string.\n\n

estatdata:information rdf:type estat:InfoType.\n
estatdata:information estat:title \"Information\"^^xsd:string.\n
estatdata:type rdf:type estat:InfoType.\n
estatdata:type estat:title \"Type\"^^xsd:string.\n\n

estatdata:lexicalunitmain rdf:type estat:LexicalType.\n
estatdata:lexicalunitmain estat:title \"Lexical Unit Main\"^^xsd:string.\n
estatdata:abbreviationallowed rdf:type estat:LexicalType.\n
estatdata:abbreviationallowed estat:title \"Abbreviation Allowed\"^^xsd:string.\n
estatdata:acronymallowed rdf:type estat:LexicalType.\n
estatdata:acronymallowed estat:title \"Acronym Allowed\"^^xsd:string.\n
estatdata:syntacticvariantallowed rdf:type estat:LexicalType.\n
estatdata:syntacticvariantallowed estat:title \"Syntactic Variant Allowed\"^^xsd:string.\n\n

estatdata:hectares rdf:type estat:MeasurementType.\n
estatdata:hectares estat:title \"Hectares\"^^xsd:string.\n
estatdata:number rdf:type estat:MeasurementType.\n
estatdata:number estat:title \"Number\"^^xsd:string.\n
estatdata:percentage rdf:type estat:MeasurementType.\n
estatdata:percentage estat:title \"Percentage\"^^xsd:string.\n
estatdata:inhabitantspersquarekilometer rdf:type estat:MeasurementType.\n
estatdata:inhabitantspersquarekilometer estat:title \"Inhabitants per Square Kilometer\"^^xsd:string.\n
estatdata:millionsofeuro rdf:type estat:MeasurementType.\n
estatdata:millionsofeuro estat:title \"Millions of Euro\"^^xsd:string.\n
estatdata:thousandsoftonnes rdf:type estat:MeasurementType.\n
estatdata:thousandsoftonnes estat:title \"Thousands of Tonnes\"^^xsd:string.\n\n

estatdata:notapplicable rdf:type estat:StatisticalType.\n
estatdata:notapplicable estat:title \"Not Applicable\"^^xsd:string.\n
estatdata:residentunit rdf:type estat:StatisticalType.\n
estatdata:residentunit estat:title \"Resident Unit\"^^xsd:string.\n
estatdata:individual rdf:type estat:StatisticalType.\n
estatdata:individual estat:title \"Individuals\"^^xsd:string.\n
estatdata:household rdf:type estat:StatisticalType.\n
estatdata:household estat:title \"Households\"^^xsd:string.\n
estatdata:accident rdf:type estat:StatisticalType.\n
estatdata:accident estat:title \"Accidents\"^^xsd:string.\n
estatdata:agriculturalholdingsandordairies rdf:type estat:StatisticalType.\n
estatdata:agriculturalholdingsandordairies estat:title \"Agricultural Holdings and/or Dairies\"^^xsd:string.\n
estatdata:geographicalarea rdf:type estat:StatisticalType.\n
estatdata:geographicalarea estat:title \"Geographical Area\"^^xsd:string.\n
estatdata:enterprise rdf:type estat:StatisticalType.\n
estatdata:enterprise estat:title \Enterprises (Corporations, Institutions)\"^^xsd:string.\n\n

estatdata:current rdf:type estat:StatusType.\n
estatdata:current estat:title \"Current\"^^xsd:string.\n
estatdata:deprecated rdf:type estat:StatusType.\n
estatdata:deprecated estat:title \"Deprecated\"^^xsd:string.\n\n
"""

f.write(helperType)

#Insert the terms
for name, row in termsEurostat.iterrows():
  f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) + " rdf:type estat:NamedEntity.\n" \
          + "estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) \
          + " estat:term \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n")
  f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) + " estat:id \"" + str(row[0]) + "\"^^xsd:integer.\n")
  
f.close()

In [10]:
##Load Article to Topics and Topics to Terms 
query =      """SELECT topic_id,term_id,probability FROM ESTAT.V1.tm_values"""

topicsTerms = load_table(cursor,query)

query =      """SELECT id,article_id,topic_id,probability FROM ESTAT.V1.tm_articles_to_topics"""

articleTopics = load_table(cursor,query)

query =      """SELECT topic_id,topic_keywords,topic_name FROM ESTAT.V1.tm_topics"""
topicsEurostat = load_table(cursor,query)

In [11]:
#Map articles to topics
helperTuple = []
for name, row in topicsEurostat.iterrows():
  helperTuple.append((row[0], row[2].replace("\"", "").replace("\'", "").replace(",", "").replace("\\", "").strip().replace(" ", "_").lower()))

f = open("termTopicType.ttl", "a")
for t in helperTuple:
  f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', t[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_").lower()) + " rdf:type estat:Topic.\n")
  f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', t[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_").lower()) + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', t[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n")
      
hashTopics = {}
for name, row in articleTopics.iterrows():
  if hashTopics.get(row[1]) == None:
      hashTopics.setdefault(row[1], []).append((row[2], row[3]))
  else:
      hashTopics[row[1]].append((row[2], row[3]))

for pr in hashTopics:
  hashTopics[pr] = sorted(hashTopics[pr], key = lambda x: float(x[1]), reverse = True)[:5]

for pr in hashTopics:
  for row in hashTopics[pr]:
    for t in helperTuple:
      if int(t[0]) == int(row[0]):
        f.write("estatdata:SEArticle" + str(int(pr)) + " estat:hasTopic estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', t[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_").lower()) + ".\n")
f.close()

###do the same for terms!!!

In [12]:
print(topicsTerms)

       topic_id  term_id  probability
0             0        0     0.000004
1             0        1     0.000290
2             0        2     0.001547
3             0        3     0.000004
4             0        4     0.000970
...         ...      ...          ...
36535        19     1822     0.000003
36536        19     1823     0.000003
36537        19     1824     0.000003
36538        19     1825     0.000003
36539        19     1826     0.000003

[36540 rows x 3 columns]


In [22]:
#Map topics to terms
helperTupleTerm = []
for name, row in termsEurostat.iterrows():
  helperTupleTerm.append((row[0], row[1].replace("\"", "").replace("\'", "").replace(",", "").replace("\\", "").strip().replace(" ", "_").lower()))

termTopics = {}
for name, row in topicsTerms.iterrows():
  if termTopics.get(int(row[0])) == None:
      termTopics.setdefault(int(row[0]), []).append((int(row[1]), row[2]))
  else:
      termTopics[int(row[0])].append((int(row[1]), row[2]))

for pr in termTopics:
  termTopics[pr] = sorted(termTopics[pr], key = lambda x: float(x[1]), reverse = True)[:20]

for pr in termTopics:
  helperList = []
  for val in termTopics[pr]:
    helperList.append(val[0])
  termTopics[pr] = helperList

print(termTopics)
f = open("termTopicType.ttl", "a")
for pr in termTopics:
  for val in termTopics[pr]:
    for top in helperTuple:
      if top[0] == pr:
          topic = top[1]
          break
    for ter in helperTupleTerm:
      if ter[0] == val:
        term = ter[1]
        break
    f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', topic).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) + \
    " estat:hasCODEDTerm estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', term).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")) + ".\n")
f.close()


{0: [255, 254, 628, 408, 1126, 219, 371, 579, 1640, 71, 1013, 151, 872, 242, 311, 426, 415, 793, 857, 1034], 1: [309, 447, 151, 203, 6, 53, 530, 297, 228, 129, 219, 199, 1270, 319, 443, 141, 128, 516, 772, 268], 2: [417, 329, 309, 1119, 187, 641, 303, 238, 160, 189, 872, 647, 530, 164, 765, 135, 1028, 383, 876, 107], 3: [121, 138, 169, 125, 156, 151, 309, 148, 114, 139, 166, 130, 347, 117, 143, 311, 161, 458, 145, 119], 4: [1028, 925, 434, 912, 1100, 159, 91, 1435, 76, 538, 1539, 348, 328, 315, 998, 1149, 1817, 18, 1712, 332], 5: [395, 647, 416, 234, 71, 302, 233, 173, 553, 437, 309, 575, 1398, 311, 91, 510, 653, 524, 295, 942], 6: [71, 242, 233, 192, 793, 371, 254, 1543, 615, 261, 1683, 699, 486, 1069, 322, 301, 255, 66, 792, 306], 7: [173, 0, 91, 41, 647, 233, 71, 29, 54, 148, 1016, 865, 107, 51, 234, 81, 6, 1572, 575, 659], 8: [254, 173, 255, 1137, 912, 695, 348, 305, 371, 1246, 339, 354, 536, 54, 1372, 253, 105, 76, 242, 350], 9: [1182, 864, 489, 442, 1193, 239, 1183, 545, 1350, 83